In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-may-2021/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-may-2021/test.csv')

In [ ]:
(train.shape,test.shape)

In [ ]:
train.target.value_counts()

In [ ]:
v_cnt ={}
for c in train.columns:
    v_cnt[c]=len(train[c].value_counts())

In [ ]:
from catboost import CatBoostClassifier, Pool ,cv

In [ ]:
train.target =train.target.str.slice(6,7).astype(int)
train.target = train.target-1 

In [ ]:
X = train.drop(columns=['target'])
Y = train.target

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_validation, y_train, y_validation = train_test_split(X, Y, train_size=0.75, random_state=42)

In [ ]:
params = {"iterations": 2,
          "depth": 5,
          "loss_function": "MultiClass",
          "learning_rate" : 0.5,
          "verbose": False}

In [ ]:
train_dataset = Pool(data=X_train,
                  label=y_train,
                  cat_features=X_train.columns)

val_dataset = Pool(data=X_validation,
                  label=y_validation,
                  cat_features=X_validation.columns)

train_pool = Pool(data=X, label=Y, cat_features=X.columns, has_header=True)


In [ ]:
from hyperopt import hp, fmin, tpe

In [ ]:
from catboost.utils import eval_metric
import catboost

In [ ]:
def hyperopt_objective(params):
    print(params)
    #model = catboost.CatBoostClassifier(**params, random_seed=42)
    #model.fit(train_dataset, verbose=10, eval_set=val_dataset,early_stopping_rounds=10)
    #y_pred = model.predict_proba(val_dataset)
    cv_data = cv(
    params = params,
    pool = train_pool,
    fold_count=5,
    shuffle=True,
    partition_random_seed=0,
    stratified=False,
    verbose=False,
        early_stopping_rounds=10
    )
    
    loss = cv_data['test-MultiClass-mean'].min()
    return loss
    #return -eval_metric(val_dataset.get_label(), y_pred[:, 1], 'MultiClass')[0]

space = {
    "iterations": 100,
     "loss_function": "MultiClass",
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.5),
    'depth': hp.randint('depth', 3, 10),
    'l2_leaf_reg': hp.uniform('l2_leaf_reg', 1, 10),
    'boosting_type': 'Plain'
}

best = fmin(hyperopt_objective,
    space=space,
    algo=tpe.suggest,
    max_evals=20,
    rstate=np.random.RandomState(seed=123))

In [ ]:
best_params = best.copy()
best_params['boosting_type'] = 'Plain' 

In [ ]:

catb_model = CatBoostClassifier(
     loss_function='MultiClass',
    random_seed=42,
    logging_level='Silent',
    iterations=100,
    boosting_type = 'Ordered',
 depth = 6,
 l2_leaf_reg = 6.60185754074968,
 learning_rate = 0.3664080258656682)

In [ ]:
catb_model.fit(
    X, Y,
    cat_features=X_train.columns,
    logging_level='Verbose',  # you can uncomment this for text output
    plot=True
);

In [ ]:
pred_df_catb=pd.DataFrame(np.round(catb_model.predict(test,prediction_type='Probability'),2))

In [ ]:
pred_df_catb.head(2)

In [ ]:
file_catb=pd.concat([test[['id']],pred_df_catb],axis=1)

In [ ]:
file_catb.columns = ['id','Class_1','Class_2','Class_3','Class_4']

In [ ]:

file_catb.to_csv('./comp_catb_holdout_hyperopt.csv',index=False)